# Neural Style Transfer with **Keras**

Hello !

This is a python script to perfom Neural Style Transfer, which consists into applying the style of a "style image" to a "content image", giving pretty results, an example :

![NST_example](https://i.imgur.com/Ch0WLH8.png)

I'm using Keras with a Tensorflow backend. 
Neural Style Transfer is an application of Deep Learning and is computer intensive, that's why I have ported my to Google Collab in order to run it on its online GPU. You can also find a version with just Python on [my github](https://github.com/Cawotte/Neural_Style_Transfer/tree/master/Python) if you want to run it on your own machine.

I won't explain how it works there, if you are interested into the math and concepts, I put some references into my github's readme file. This is just to use it. If you are interested into the code, you can take a look at neural_style_transfer.py, it's commented. 


Follow my instructions to run this code and try it for yourself !

**The setup : **

You must have :

* **This very file**, it's a python script for Google Colab, put it on your Google Drive and open it with Colab.
*  **neural_style_transfer.py**, it's the file with the important functions.
*  An **images.zip** archives, with the images you want to use, in the file and folder hierarchy that you want, as long as you know it. We use a zip instead of a folder because it's faster to upload.

You can get all of them on [my github](https://github.com/Cawotte/Neural_Style_Transfer/tree/master/Colab). There's also an images.zip with a bunch of images to toys with.

You will be able to upload these files to Colab either through Google Drive, or from your PC.
The method from PC is very straightforward but takes more time to upload, Google Drive is faster but quite bothersome.



### Authentification

First, run the below cell to connect to Google Colab, open a session, import useful packages, and be able to import file froms Google Drive. 

It will ask you to open a browser link twice to get a key to authentificate yourself, that you'll have to copy in an input bar.

In [0]:

#Install basic libraries + authorizations

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#Libraries and imports to use Google Drive and Google SDK
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
import pandas as pd
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# **UPLOAD THE FILES**

### 1 - with Google Drive

First, put all the files in the same folder on Google Drive.

In order to access your Google Drive files and upload them to Colab (Colab has its own working directory that's is NOT the current drive folder !) you'll have to know their files ID. Several solutions :

* Run the following cell, if will show you the id of each files and folders in your root Google Drive directory. If those files are not into your root Directory, just replace 'root' by the ID of the folder they are in, and repeat the processus until you find the files.
* Click on the file on your Google Drive, and get a shareable link, the part in **bold** in this example correspond to the part of the URL that correspond to the file ID : https//drive.google.com/file/d/**1YLoXGN2M7h5dm1oqo7JYNm3p5HxFf6_7**/view?usp=sharing


In [0]:
#Search for files ID.

file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))
  
#That's my own ID I just don't want to have to look for them again, you can ignore them.
  #1MO0Ul6kfHaBbAnKp-rzjP93PjFVarIFM NST Folder ID
    #1YLoXGN2M7h5dm1oqo7JYNm3p5HxFf6_7 neural_style_transfer.ipnyb #this
    #1kOOTgh4bNak5CxUBiH2txcTw-TaUqvgp images.zip

Now that you know their id, modify the next cell with those values and run it to upload them.
* **image_zip_id** is the id of the images.zip file.
* **nst_py_id** is the id of the neural_style_transfer.py file.

In [0]:
#Complete/replace with your files ID.
image_zip_id = '1kOOTgh4bNak5CxUBiH2txcTw-TaUqvgp';
nst_py_id = '1YLoXGN2M7h5dm1oqo7JYNm3p5HxFf6_7';

### Upload the images 

#Download images from Drive to Colab's working directory.
download = drive.CreateFile({'id': image_zip_id})
download.GetContentFile('images.zip')

#Unzip the images folder in the Colab local directory, remember your folder hierarchy !
!unzip -o images.zip > /dev/null #Hide output

#Display Colab working directory, you are supposed to see
# 'datalab', 'images', 'images.zip', 'neural_style_transfer.py'

### Upload the script and import it as a module.
download = drive.CreateFile({'id': nst_py_id})
download.GetContentFile('neural_style_transfer.py')

!ls #Show Colab's current working directory, you should see both files in it.


### 2 - Upload from your PC

Just run the code below, it will open the file explorer and you'll just need to select your two files.
It's way more simple than Google Drive, but uploading takes more time, especially if you have a lot of images.

In [0]:
from google.colab import files
src = list(files.upload().values())[0]

!ls #Show Colab's current working directory, you should see both files in it.


##Once Uploaded

Now that those files are uploaded, we can unzip the zip with the images files and import the python script as a module wit the code below.

In [0]:
#Unzip the images folder in the Colab local directory, remember your folder hierarchy !
!unzip -o images.zip > /dev/null #Hide output

!ls
#Display Colab's current working directory, you are supposed to see
#'images', 'images.zip', 'neural_style_transfer.py', and some additional unrelated stuff

#Import the magical function from our script.
from neural_style_transfer import style_transfer



### Before running the code

By default, Google Colaboratory use a CPU. If it's the first time you are opening and using this file in Colab, you need to enable the GPU.

Go to **Edit > Notebook settings or Runtime > Change runtime type** and select GPU as Hardware accelerator.

You can run the code cell below to check that the GPU is used.

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# **Now you can try it out yourself !**

You completed the most annoying part ! Now, you can adjust all the different possible parameters in the below cell, then launch the transformation with the next cell !

A rundown of all the parameters :

### IMAGES PATH
The name of the images used for your transformation
* **content_img_name** : The content image name
* **style_img_name** : The style image name
* **output_img_name** : The name of the output image

#### Full paths
* **style_image_path**
* **content_image_path**
* **output_image_path**

Complete those with the directory path of where you can find those images . (That's why you need to remember the hierarchy of your images folder)


### DIMENSIONS
To control the size of your output image. The bigger it is, the more time it takes to compute.
* **loadDims** : If set to True, use the native dimensions of the content image, else, it will resize with the width and height parameters values.
* **width** and **height** : Dimensions to which resize the image if loadDims is set to False.
* **rescale** : If loadDims is set to True, will resize the image by this factor.
* **max_size** : If loadDims is set to True, while automatically resize the image to below this value if it has more pixels than max_size. Set to -1 to ignore.

### MISC

* **withBaseImage** : If True, will use the content image as the starting image when starting the transformation. Set to False to start from random noises. I recommend True as I hardly had any convincing results with noises.
* **stepsBeforeSaveAndShow** : The number of iterations before it save and show the current generated image, for example if set to 3, will display the current output image and save it every 3 iterations.
* **nb_iterations** : Total number of iterations, I recommend between 20-50. Below is not always reliable and above hardly shows any improvements. 

### LOSSES WEIGHTS
Related to the calculation for the algorithms. It tells how much the generated image needs to be close to the content and style images in term of content and style. 

Usually, content_weight is very very low compared to the style_weight. 
The variation weight is used to have more smooth looking pictures, there's barely any needs to modify it from it default value.
* **content_weight** : 0.025 as default value.
* **style_weight** : 5.0 as default value.
* **var_weight** : 1.0 as default value.

### LAYERS USED FOR LOSSES CALCULATIONS
Also related to the loss calculation. I won't go into the Math, but this algorithm put the images into a pre-trained Convolutional Neural Network (VGG16 in our case, it's quite efficient), and retrieve the outputs on some of its layers to perform calculation and tell how close our generated image is close to the wanted result :

Content loss tells how much the generated image is close to the content image in term of content, and usually only use the output of a single layer, one of the latest of the neural network.

Style loss tells how much the generated image is close to the style image in term of style, and usually only use several layers outputs, often the first one from each "block" of convolutional layers. 

Those values are the list of the names of the layers used for each of those calculations. You need to know the layers name to modify it properly
* **ln_content**
* **ln_style**



In [0]:

### IMAGES PATH

#Name of the images used for the style transfer.
content_img_name = "sunflower.jpg"
style_img_name = "stained_glass_1.jpg"
output_img_name = "glass_sunflower.jpg"

#Full path of the image used, complete with your image folder path
style_image_path = "./images/" + style_img_name
content_image_path =  "./images/" + content_img_name
output_image_path = "./output/" + output_img_name


### DIMENSIONS
#Dimensions of the output image. Ignore is loadDims is set to True.
width = 500;
height = 500;
#True if you want to use native content image dims.
loadDims = True; 
#Value by which rescaling the image, used only if LoadDims is set to True.
rescale = 1.0
#Scale the image down if height*width > max_size, keeps the image size ratio, set to -1 to ignore.
#Used only with LoadDims = true.
max_size = 250*250

### MISC

#True if starting from content image, False for random noise as starting image.
withBaseImage = True; #Recommended to leave it on True
#Number of iterations before saving and showing the current generated image, -1 to ignore.
stepsBeforeSaveAndShow = 3
#Number of iterations on which calculating losses and modifying the base image. 
nb_iterations = 20  #between 20-50 recommended.

### WEIGHTS : Importance of each kind of losses. 
#default values : content = 0.025, style = 5.0, var_w = 1.0
content_weight = 0.025
style_weight = 5.0
var_weight = 1.

### Layer used for the Loss calculations
ln_content = ["block5_conv2"]
ln_style = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]

Now run the next cell to start the transformation !

In [0]:
style_transfer(
        content_image_path, #Path of the content image
        style_image_path, #Path of the style image
        output_image_path, #Path of the output image
        loadDims, #True if using native image dim
        withBaseImage, #True if using content image as starting image, false for random noises.
        rescale, #Rescale the image, only useful if loadDims = true
        max_size, #Automatically rescale if height*width > max_size. -1 to ignore.
        height, #wanted height of the images, useless if loadDims.
        width, #wanted width of the images, useless if loadDims.
        nb_iterations, #number of iterations
        stepsBeforeSaveAndShow, 
        content_weight,
        style_weight,
        var_weight,
        ln_content, #Layers used for loss content calculation
        ln_style #Layers used for style content calculation.
        )


You can download your output image by running the code cell below !

You can also download a previously generated image by changing "output_image_path" by the path of that image in the Colab directory.

In [0]:
#Download the output image to your PC.
from google.colab import files
files.download(output_image_path)

**WARNING**

The next cell's code is used to reset the session, by killing it. I got sometimes bug when I tried to re-upload an image folder when I wanted to add new images, so I used it to just restart the whole session from the beginnin. You'll have to authentificate again.

In [0]:
#COMPLETELY KILL AND RESET THE SESSION
#Sometime there's bug when you try to re-upload your images.zip in the same session (to add new images), 
#You can use it to totally reset the session, you'll have to authentificate again !
!kill -9 -1

In [0]:
#Clean the working directory, delete all images file, folder and zip. 
#Useful if you want to delete previous images or changed your files hierarchy.
!rm -r images
!rm images.zip

!ls